## AFS 01 - Sales Report

This is a number of repors covering AFS_01 - A sales forecasting model using DBT and Clearscape


Database hierarchy is
CS01
CS01_RAW
CS01_RAW_VIEW
CS01_TARGET
CS01_TARGET_VIEW

In [1]:
import teradatasql
import yaml
import sys
import os
import sqlparse
import pandas as pd



THis will get the DBT config so we can use the same logon

In [2]:
# get home directory
from os.path import expanduser
import pprint
home = expanduser("~")
# assume project is called dbt_clearscape
project = 'dbt_clearscape'
# read config file
with open(home + '/.dbt/profiles.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
# get project config
if project not in config:
    print('project not found in config file')
project_config = config[project]
# get target
target = project_config['target']
# get connection details
project_connection = project_config['outputs'][target]
#pprint.pprint(project_connection)


This will connect to teradata using the DBT configuaration

In [4]:
# connect to teradata
conn = teradatasql.connect(host=project_connection['server'], user=project_connection['username'], password=project_connection['password'])    

List all object in databases , any object in DB CS01% and table Prefixed with AFS_01

In [ ]:
df_tables = pd.read_sql("""select databasename,tablename from dbc.tablesv where databasename  LIKE 'CS01%' and tablename LIKE 'AFS_01%' order by 1,2""", conn)
df_tables

C:\Users\cw171001\AppData\Local\Temp\ipykernel_29884\1171293227.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql("""select databasename,tablename from dbc.tablesv where databasename  LIKE 'CS01%' and tablename LIKE 'AFS_01%' order by 1,2""", conn)


,DataBaseName,TableName
0,CS01,AFS_01_Resample
1,CS01_TARGET,AFS_01_ArimaForecast
2,CS01_TARGET,AFS_01_AutoArima
3,CS01_TARGET,AFS_01_AutoArima_Forecast
4,CS01_TARGET,AFS_01_Dickey_Fuller_After
5,CS01_TARGET,AFS_01_Dickey_Fuller_Before
6,CS01_TARGET,AFS_01_Resample
7,CS01_TARGET,AFS_01_SalesData_Outlier
8,CS01_TARGET,AFS_01_SeasonalNormalize
9,CS01_TARGET_VIEW,AFS_01_ArimaARTFITRESIDUALS


In [ ]:
AFS_01_ArimaARTFITRESIDUALS
AFS_01_AutoArima
AFS_01_AutoArima_FIT_RESIDUALS
AFS_01_Dickey_Fuller_After
AFS_01_Outlier_FIT
AFS_01_Resample
AFS_01_SeasonalNormalize


# Create all the database

The databases are

- CS01
- CS01_RAW
- CS01_RAW_VIEW
- CS01_TARGET
- CS01_TARGET_VIEW



In [5]:
# TODO - setup parameterised db config
db_list = {}
db_list ['CS01'] = {'owner': {project_connection['username']},'space':0}
db_list ['CS01_RAW'] = {'owner': 'CS01','space':0}
db_list ['CS01_RAW_VIEW'] = {'owner': 'CS01','space':0}
db_list ['CS01_TARGET'] = {'owner': 'CS01','space':10000}
db_list ['CS01_TARGET_VIEW'] = {'owner': 'CS01','space':0}

In [9]:
# Delete and then drop all databases
with conn.cursor() as csr:
    for i,db in enumerate(db_list):
        # It is sometime very slow in deleting databases so drop each table and view first
        with conn.cursor() as csr2:
            csr2.execute(f"""SELECT tablename FROM DBC.tablesv WHERE DATABASENAME = '{db}' AND tablekind in ('T','O')""")
            # copy list of tables in table_list
            table_list = csr2.fetchall()
            for table in table_list:
                print(f'DROP TABLE {db}.{table[0]}')
                csr2.execute(f'DROP TABLE {db}.{table[0]}')
            csr2.execute(f"""SELECT tablename FROM DBC.tablesv WHERE DATABASENAME = '{db}' AND tablekind in ('V')""")
            # copy list of views in view_list
            view_list = csr2.fetchall()
            for view in view_list:
                print(f'DROP VIEW {db}.{view[0]}')
                csr2.execute(f'DROP VIEW {db}.{view[0]}')
        print(i,db)
        print(f'DELETE DATABASE {db}')
        csr.execute(f'DELETE DATABASE {db}',ignoreErrors=[3807,3802])
        print(f'GIVE {db} TO {project_connection["username"]}')
        csr.execute(f'GIVE {db} TO {project_connection['username']}',ignoreErrors=[3807,3802])
with conn.cursor() as csr:
    for i,db in enumerate(db_list):
        print(i,db)
        print(f'DROP DATABASE {db}')
        csr.execute(f'DROP DATABASE {db}',ignoreErrors=[3807,3802])

0 CS01
DELETE DATABASE CS01
GIVE CS01 TO demo_user
1 CS01_RAW
DELETE DATABASE CS01_RAW
GIVE CS01_RAW TO demo_user
2 CS01_RAW_VIEW
DELETE DATABASE CS01_RAW_VIEW
GIVE CS01_RAW_VIEW TO demo_user
3 CS01_TARGET
DELETE DATABASE CS01_TARGET
GIVE CS01_TARGET TO demo_user
4 CS01_TARGET_VIEW
DELETE DATABASE CS01_TARGET_VIEW
GIVE CS01_TARGET_VIEW TO demo_user
0 CS01
DROP DATABASE CS01
1 CS01_RAW
DROP DATABASE CS01_RAW
2 CS01_RAW_VIEW
DROP DATABASE CS01_RAW_VIEW
3 CS01_TARGET
DROP DATABASE CS01_TARGET
4 CS01_TARGET_VIEW
DROP DATABASE CS01_TARGET_VIEW


In [10]:
# TODO : Convert to a more parameteris version

In [11]:
sql = f"CREATE DATABASE CS01 FROM {project_connection['username']} AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01 AS 'Base database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [12]:
sql = f"CREATE DATABASE CS01_RAW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_RAW AS 'raw database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [13]:
sql = f"CREATE DATABASE CS01_RAW_VIEW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_RAW_VIEW AS 'raw database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [14]:
sql = f"CREATE DATABASE CS01_TARGET FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_TARGET AS 'target database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [15]:
sql = f"CREATE DATABASE CS01_TARGET_VIEW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_TARGET_VIEW AS 'target database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [17]:
# grant intra database access
sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_RAW TO CS01_RAW_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_TARGET TO CS01_TARGET_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_RAW_VIEW TO CS01_TARGET_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

# dbc access
sql = f"GRANT SELECT ON DBC TO CS01_RAW_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')


# val access
sql = f"GRANT SELECT ON VAL TO CS01_RAW  WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')


Grant created
Grant created
Grant created
Grant created
Grant created


Allocate space for the databases

In [18]:
with conn.cursor() as csr:
    # drop space database if it exists
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [3802])
    # create space data with space
    csr.execute(f'CREATE DATABASE CS01_SPACE FROM {project_connection['username']}  AS PERM=200000000 ')
    # give space to database
    csr.execute(f'GIVE CS01_SPACE TO CS01_RAW')
    # drop space
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [])

In [19]:
with conn.cursor() as csr:
    # drop space database if it exists
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [3802])
    # create space data with space
    csr.execute(f'CREATE DATABASE CS01_SPACE FROM {project_connection['username']}  AS PERM=200000000 ')
    # give space to database
    csr.execute(f'GIVE CS01_SPACE TO CS01_TARGET')
    # drop space
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [])